In [2]:
from twython import Twython, TwythonError
import pandas as pd
import csv
import math

In [4]:
API_key = "M3ptGiuDD8KoIldEFCAkImEGi"

API_secret_key = "qvTDEypEKXOpzuFCpY9DrPu4OTItPgzOA6OXAbU0ToYz4P2X0E"

In [5]:
twitter = Twython(API_key, API_secret_key)

# Question 1 :
#### Afficher la date, le nom d'utilisateur et le contenu du tweet ayant l'ID : 1157345692517634049 (0.5 Pts)

In [7]:
test_id = "1157345692517634049"

In [17]:
tweet = twitter.show_status(id=test_id)
print("Date :\t\t{}\nUser :\t\t{}\nContent :\t{}".format(tweet["created_at"], tweet["user"]["name"], tweet["text"]))

Date :		Fri Aug 02 17:41:30 +0000 2019
User :		Donald J. Trump
Content :	A$AP Rocky released from prison and on his way home to the United States from Sweden. It was a Rocky Week, get home ASAP A$AP!


# Question 2 :
#### Implémenter la fonction `hydrate_database` (3.5 Pts)

*Attention : Il faut également gérer le cas où la feature demandée n'est pas une clé du dictionnaire mais une "sous-clé", comme c'est le cas pour le nom d'utilisateur par exemple (accessible dans la feature *user*, qui lui même est un dictionnaire). Un moyen simple pour pallier à ce problème consiste à considérer la feature comme une liste, qui contiendrait la clé et les sous-clés si il y a lieu (voir exemple plus bas)

*Indice : La fonction `sleep` du module time permet de patienter le temps nécessaire*

In [113]:
def hydrate_database(source, destination, twitter, n_tweets=100):
    """ Takes a csv file of tweet_ids and assiociated labels, recover the tweet content using a Twython object
        and writes it in the destination file in the csv format with columns : 
            tweet_id, label, tweet_content
        If the tweet is retweeted, we recover the original tweet, since the retweeted version might have been truncated.
    """
    
    df = pd.read_csv(source)
    labels = dict(zip(df["tweet_id"].values, df["label"].values))
    destination = open(destination, "w")
    destination.write("tweet_id\tlabel\ttweet_content\n")
    entries = 0
    retrieved = 0
    for i in range( math.ceil( len(df)/n_tweets )):
        imin = i * n_tweets
        imax = min( len(df), (i+1) * n_tweets )
        entries += imax-imin
        print("Retrieved:\t{:5d} /{:5d}".format(retrieved, entries),end='\r', flush=True)
        try :
            tweets = twitter.lookup_status(id=list(df["tweet_id"].values[imin:imax]))
        except TwythonError as e:
            print("Error", e.error_code)
            continue
        retrieved += len(tweets)
        for tweet in tweets:
            tweet_id = tweet["id"]
            if "retweeted_status" in tweet.keys():
                tweet = tweet["retweeted_status"]
            destination.write("{}\t{}\t{}\n".format(tweet["id"], labels[tweet_id], '\"'+repr(tweet["text"])[1:-1].replace('\t',' ')+'\"' ))

    destination.close()

In [114]:
hydrate_database("data/hatespeech_labels.csv", "data/test.csv", twitter)